In [1]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import pvlib
sys.path.append(os.getcwd())
from persistence import PVForecast


In [25]:
def get_s_persistance(ghi, horizon, freq='10s', location=None):
    """
    get a single persistance forecast
    """

    if location is None:
        location = pvlib.location.Location(latitude=59.973218,
                                       longitude=11.051604,
                                       altitude=130,
                                       tz='CET')

    persistance = PVForecast(measured_signal=ghi, 
                             binning=None,
                             f_h=horizon,
                             s_h = '0s',
                             f_i=freq,
                             location=location)
                             
    sp = persistance.smart_persistance()     

    return sp

def get_clearsky2(location, index, G_0=1360, max_sza=90, kind='clear') -> pd.Series:
    """
    Calculate clearness.
    
    :param G_0: float
        the extra terrestrial radiation received by the Earth from the Sun
    :return: G_clear : pandas.Series
    
    """
    
    # Get solar zenith angle (SZA):
    sza = location.get_solarposition(index)['zenith']
    
    

    # Get clearsky
    if kind=='clearsky':
        G_clear = location.get_clearsky(sza.index)
    elif kind == 'clear':
        # Calculate cos(SZA):
        cos_zenith = pvlib.tools.cosd(sza)
    else:
        print('select clear or clearsky, default to clear')
        # Calculate cos(SZA):
        cos_zenith = pvlib.tools.cosd(sza)
    
    # Calculate clearness:
    G_clear = G_0 * np.maximum(cos_zenith, np.cos(np.radians(max_sza)))
    
    return G_clear


def get_clearsky(ghi, horizon, freq='10s', location=None):
    """
    get a single persistance forecast
    """

    if location is None:
        location = pvlib.location.Location(latitude=59.973218,
                                       longitude=11.051604,
                                       altitude=130,
                                       tz='CET')

    persistance = PVForecast(measured_signal=ghi, 
                             binning=None,
                             f_h=horizon,
                             s_h = '0s',
                             f_i=freq,
                             location=location)
                             
    GHI_clear = persistance.get_clearness()     

    return GHI_clear

    
    
def get_spersistance_df(index, horizon_steps=90, location=None, freq='10s', ghi_data_path=None, ghi_df=None,revnorm=False):
    """
    get datafram of persistance forecasts, where the index indicates the time at which the persistance forecast is issued
    With a given frequency
    A given amount of steps forward in time
    """
    if ghi_data_path == None:
        ghi_data_path = 'Data_GHI/KZPR_IRHoSunP'
        
    # dates = np.sort(list(set(pd.to_datetime(index).date)))
    dates = np.sort(list(set(index.dt.date)))
    #columns = [(i*10)+10 for i in range(horizon_steps)]
    sp_df = pd.DataFrame()

    for day in dates:
        print(day)
    
        day_ind = "".join(map(lambda x: x, str(day).split("-")))

        if ghi_df is None:    
            if (ghi_data_path.split('/')[-1].split('_')[0] == 'processed'):
                ghi = (pd.read_pickle(f'{ghi_data_path}/{day_ind}.pkl').tz_localize('CET')+1)*(1367/2)
            else:
                ghi = pd.read_pickle(f'{ghi_data_path}/{day_ind}/{day_ind}.pkl').set_index('date_time').tz_localize('CET')
        else:
            ghi = ghi_df[ghi_df.index.date == day]
            
            if revnorm==True:
                ghi = (ghi + 1)*(1367/2)
    
        if location is None:
            location = pvlib.location.Location(latitude=59.973218,
                                           longitude=11.051604,
                                           altitude=130,
                                           tz='CET')
    
        # first horizon
        sp = get_s_persistance(ghi=ghi, horizon=freq, freq=freq, location=location)

        # rest of horizons
        for step in range(2,horizon_steps+1):
            h_seconds = step*10
            sp_step = get_s_persistance(horizon=f'{h_seconds}s', freq=freq, ghi=ghi, location=location)
        
            sp = pd.concat([sp,sp_step],axis=1)

        # Shift forecast to correspond to issued timestamp
        sp.columns = [i*10 for i in range(1,horizon_steps+1)]
        for i,col_name in enumerate(sp.columns):
            sp.iloc[:-i-1,i] = sp.iloc[i+1:,i]
        
        # concat each day
        sp_day = sp.loc[index[index.dt.date == day],:]
        sp_df = pd.concat([sp_df, sp_day], axis=0)
    return sp_df


In [23]:
# file_path_ghi = "C:\\Users\\nikla\\Documents\\phd\\paper2\\code\\IFE_dataset_model\\irr_df_20210622_20240618.pkl"
# irr_df = pd.read_pickle(file_path_ghi)

path = 'C:\\Users\\nikla\\Documents\\phd\\paper2\\code\\IFE_dataset_model\\trainval_df_preprocessed.pkl'
trainval_df = pd.read_pickle(path)
ghi = trainval_df['GHI']

index = pd.read_pickle('C:\\Users\\nikla\\Documents\\phd\\paper2\\code\\IFE_dataset_model\\trainval_C_index.pkl')

index_filtered = index.iloc[::3]
ghi_rolling = ghi.rolling(window=3).mean()#[index_filtered]


In [21]:
ghi_rolling

2022-06-09 05:57:00+02:00    52.000004
2022-06-09 05:57:30+02:00    55.666665
2022-06-09 05:58:00+02:00    58.999994
2022-06-09 05:58:30+02:00    62.666655
2022-06-09 05:59:00+02:00    66.999982
                               ...    
2024-02-29 15:48:40+01:00     7.999988
2024-02-29 15:49:10+01:00     8.333321
2024-02-29 15:49:40+01:00     8.999987
2024-02-29 15:50:10+01:00     8.999987
2024-02-29 15:50:40+01:00     8.999987
Name: GHI, Length: 37090, dtype: float64

In [ ]:

pers = get_spersistance_df(index_filtered, horizon_steps=30, location=None, freq='30s', ghi_data_path=None, ghi_df=ghi_rolling)



2022-06-09
2022-06-10


KeyError: "None of [DatetimeIndex(['2022-06-10 05:51:10+02:00', '2022-06-10 05:51:40+02:00',\n               '2022-06-10 05:52:10+02:00', '2022-06-10 05:52:40+02:00',\n               '2022-06-10 05:53:10+02:00', '2022-06-10 05:53:40+02:00',\n               '2022-06-10 05:54:10+02:00', '2022-06-10 05:54:40+02:00',\n               '2022-06-10 05:55:10+02:00', '2022-06-10 05:55:40+02:00',\n               ...\n               '2022-06-10 20:35:10+02:00', '2022-06-10 20:35:40+02:00',\n               '2022-06-10 20:36:10+02:00', '2022-06-10 20:36:40+02:00',\n               '2022-06-10 20:37:10+02:00', '2022-06-10 20:37:40+02:00',\n               '2022-06-10 20:38:10+02:00', '2022-06-10 20:38:40+02:00',\n               '2022-06-10 20:39:10+02:00', '2022-06-10 20:39:40+02:00'],\n              dtype='datetime64[ns, CET]', length=1778, freq=None)] are in the [index]"

: 

In [ ]:
pers.to_pickle('persistence_30s.pkl')